Methods 5 - Logistic Regression
-------------------------------
5.3.2020  
Mathematics and Methods in Machine Learning and Neural Networks    
Helsinki Metropolia University of Applied Sciences

The aim of this exercise is to predict spinal conditions (_disk hernia_, _spondylolisthesis_) or absence thereof, based on radiographic measurements.

The prediction method used is _multinomial logistic regression_.

Data source: http://archive.ics.uci.edu/ml/datasets/Vertebral+Column

Additional reference: http://www.oref.org/docs/default-source/default-document-library/sdsg-radiographic-measuremnt-manual.pdf

In [63]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

We do multiclass prediction, so using the 3-class data set.

In [64]:
url = r'http://users.metropolia.fi/~simomake/coursework/ml/column_3C.dat'
names = ['pelvic incidence', 'pelvic tilt', 'lumbar lordosis angle', 'sacral slope', 'pelvic radius', 'grade of spondylolisthesis', 'condition']
df = pd.read_csv(url, 
                 sep = ' ', 
                 index_col = None,
                 names = names)
df.head()

,pelvic incidence,pelvic tilt,lumbar lordosis angle,sacral slope,pelvic radius,grade of spondylolisthesis,condition
0,63.03,22.55,39.61,40.48,98.67,-0.25,DH
1,39.06,10.06,25.02,29.00,114.41,4.56,DH
2,68.83,22.22,50.09,46.61,105.99,-3.53,DH
3,69.30,24.65,44.31,44.64,101.87,11.21,DH
4,49.71,9.65,28.32,40.06,108.17,7.92,DH


In [65]:
# re-encode condition column
target_names = ['NO', 'SL', 'DH']
df['condition'].replace(target_names, [0,1,2], inplace=True)

In [66]:
# split into explanatory and response variables 
X = df.drop(['condition'], axis=1)
Y = df['condition']

In [67]:
# build and fit model
reg = LogisticRegression(solver='lbfgs', multi_class='ovr')
reg.fit(X,Y)

print("Coefficients: ",reg.coef_)
print("Intercept: ", reg.intercept_)

# compute predicted values from training set
Y_pred = reg.predict(X)

cm = confusion_matrix(Y, Y_pred)
print("Confusion matrix:\n",cm)

accuracy = (cm[0][0]+cm[1][1]+cm[2][2])/(cm.sum())
print("Accuracy calculated from the training set = %.3f" % (accuracy))

print(classification_report(Y, Y_pred, target_names=target_names))

Coefficients:  [[ 0.03205038 -0.10757546  0.01869301  0.06459006  0.10677264 -0.16808262]
 [ 0.05461569  0.0083767  -0.01056203  0.04465278 -0.01597843  0.29990437]
 [-0.02629018  0.12673318 -0.04116507 -0.12509988 -0.10980262 -0.0998079 ]]
Intercept:  [-15.15571758  -7.80412803  18.00060525]
Confusion matrix:
 [[ 85   2  13]
 [  3 146   1]
 [ 19   1  40]]
Accuracy calculated from the training set = 0.874
              precision    recall  f1-score   support

          NO       0.79      0.85      0.82       100
          SL       0.98      0.97      0.98       150
          DH       0.74      0.67      0.70        60

    accuracy                           0.87       310
   macro avg       0.84      0.83      0.83       310
weighted avg       0.87      0.87      0.87       310



The model is __good at detecting spondylolisthesis__ at 97 % sensitivity and 98 % precision. This is because `grade of spondylolisthesis` is given as an input feature.

The model detects an existing __disk hernia__ at a 67 % probability (sensitivity). A patient classified by the model as having a disk hernia actually has disk hernia at a 74 % probability (precision).

For a __normal spine__, the sensitivity and precision are 85 % and 79 %, accordingly.

The above statistics were calculated against training data. To get more reliable results, they should be recalculated using previously unseen data (or cross-validation).

In [68]:
# cross-validate
k = 10  # number of folds
scores = cross_val_score(estimator=reg,
                        X=X,
                        y=Y,
                        scoring="accuracy",
                        cv=k)
print("Accuracies from %d individual folds:" % k)
print(scores)
print("Accuracy calculated using %d-fold cross validation = %.3f" % (k, scores.mean()))

Accuracies from 10 individual folds:
[0.74193548 0.80645161 0.90322581 0.67741935 0.93548387 0.93548387
 0.90322581 0.87096774 0.93548387 0.87096774]
Accuracy calculated using 10-fold cross validation = 0.858


The __overall accuracy__ of the model is __85.8 %__. Cross validation accuracy is only about 1 % weaker than training set accuracy, so we shouldn't be overly concerned with overfitting.

In [82]:
# get estimated probabilities for a single sample

input_sample = [39.36, 7.01, 37, 32.35, 117.82, 1.9] # NO

prob = reg.predict_proba(np.array(input_sample).reshape((1,-1))).round(3)
res = pd.DataFrame(prob, columns=target_names)

print("Probability of vertebral abnormality:", (1-res['NO'][0]).round(3))
print("\nClass probabilities:")
res

Probability of vertebral abnormality: 0.337

Class probabilities:


,NO,SL,DH
0,0.663,0.003,0.334
